In [1]:
from JupyterDock.Preparation import SanitizeProtein, SanitizeLigands, fetch_protein_from_pdb

from JupyterDock.Docking import Docker

from JupyterDock.Analysis import VisualizeDocking

/home/angel/miniconda3/envs/JupyterDock/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [ ]:
fetch_protein_from_pdb(entry="1NDZ")

In [ ]:
ProteinSanitizer=SanitizeProtein(protein_file='files/1NDZ_clean.pdb')

In [ ]:
ProteinSanitizer.pdbfixer_sanitization()

In [ ]:
LigandSanitizer=SanitizeLigands('files/1NDZ_lig.mol2')

In [ ]:
LigandSanitizer.run_sanitization()

In [ ]:
LigandSanitizer.ligands[0]

In [2]:
RunDocking=Docker(receptor_file='files/1NDZ_clean_sanitized.pdb',ligands_file='files/1NDZ_lig_sanitized.mol2',selection='lig',extending=4)

In [ ]:
RunDocking.run_smina()

In [ ]:
RunDocking.smina_params

In [ ]:
RunDocking.run_ledock()

*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 1)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 3)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 4)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PD

In [ ]:
import os,re

from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import AllChem

In [ ]:
tmp_dir='_dock_tmp/'

In [ ]:
template=Chem.MolFromMol2File('_dock_tmp/1AZ8.mol2',removeHs=True)
template

In [ ]:
out_results=pybel.Outputfile(filename=f"files/babel.sdf",format="sdf",overwrite=True)

for dok_file in os.listdir(tmp_dir):
    if 'dok' in dok_file:
        with open(os.path.join(tmp_dir,dok_file), 'r') as f:
            doc=f.readlines()
            
    
        start=[index for (index,p) in enumerate(doc) if 'REMARK Cluster' in p]
        end=[index-1 for (index,p) in enumerate(doc) if 'END' in p]

        interval=list(zip(start,end))
    
        for index,pair in enumerate(interval):
            block = ",".join(doc[pair[0]:pair[1]]).replace(',','')
            
            mol=Chem.MolFromPDBBlock(block,removeHs=True,proximityBonding=True)
            newMol = AllChem.AssignBondOrdersFromTemplate(template, mol)
            print(index,newMol)
            
            #m=pybel.readstring(format='pdb',string=block,)
            #print(m.data)
            #m.title = dok_file.split('/')[-1].split('.')[0]
            #m.data.update({'Pose':index+1})
            #m.data.update({'Score':m.data['REMARK'].split()[6]})
            #del m.data['REMARK']
            #out_results.write(m)
        
    #out_results.close()

In [ ]:
newMol

In [ ]:
mols[1]

In [ ]:
m.data['REMARK'].split()[1]

In [ ]:
Chem.MolToSmiles(mm)

In [ ]:
out_results=pybel.Outputfile(filename=f"{self.ligands_file.split('.')[0]}_ledock_results.{self.out_format}",format=self.out_format,overwrite=True)
        


        

        

        

        for num,i in enumerate(interval):
            block = ",".join(doc[i[0]:i[1]]).replace(',','')


            m=pybel.readstring(format='pdb',string=block)

            m.title = dok_file.split('/')[-1].split('.')[0]
            m.data.update({'Pose':m.data['REMARK'].split()[4]})
            m.data.update({'Score':m.data['REMARK'].split()[6]})

            del m.data['REMARK']

            out_results.write(m)

out_results.close()

In [ ]:
view=VisualizeDocking(receptor='files/1AZ8_clean.pdb',ligands='files/1AZ8_lig_sanitized_ledock_results.sdf',reference='files/1AZ8_lig_sanitized.mol2',pose=1)

In [ ]:
v=view.view()

In [ ]:
v.show()

In [ ]:
import MDAnalysis as mda
from MDAnalysis.coordinates import PDB

import prolif as plf
from prolif.plotting.network import LigNetwork


In [ ]:
# load protein
prot = mda.Universe("files/1AZ8_clean_sanitized.pdb")
prot = plf.Molecule.from_mda(prot)
prot.n_residues

In [ ]:
lig_suppl = list(plf.sdf_supplier('files/1AZ8_lig_sanitized_ledock_results.sdf'))

In [ ]:
fp = plf.Fingerprint()

In [ ]:
fp.run_from_iterable(lig_suppl, prot)

In [ ]:
net = LigNetwork.from_ifp(results_df,lig_suppl[0],kind="frame", frame=0,rotation=270)
net.display()